# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [16]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [17]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Re-capitalize city names, since the API brought them in lowercase
city_data_df["City"] = city_data_df["City"].str.title()

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Trang,7.5563,99.6114,82.31,78,20,2.30,TH,1722913341
1,1,Port-Aux-Francais,-49.3492,70.2194,34.27,96,100,29.55,TF,1722912830
2,2,Kapa'A,22.0752,-159.3190,81.48,67,25,7.00,US,1722913342
3,3,Cadale,2.7603,46.3222,75.87,76,100,17.00,SO,1722913342
4,4,Thompson,55.7433,-97.8635,66.38,48,75,3.44,CA,1722913204


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [18]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1.0,
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [19]:
# Narrow down cities that fit criteria and drop any results with null values
# Criteria max temp between 68°F and 78°F, wind speed of at most 10 mph, and
# 0% cloudiness.
ideal_df = city_data_df.loc[
    (city_data_df["Max Temp"] >= 68) &
    (city_data_df["Max Temp"] <= 78) &
    (city_data_df["Wind Speed"] <= 10) &
    (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
ideal_df = ideal_df.dropna()

# Display sample data
ideal_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
68,68,Remire-Montjoly,4.9167,-52.2667,77.04,100,0,0.00,GF,1722913370
104,104,Moranbah,-22.0016,148.0466,76.41,40,0,2.30,AU,1722913385
219,219,Valguarnera Caropepe,37.4953,14.3903,73.45,81,0,1.16,IT,1722913430
225,225,Ha'Il,27.5219,41.6907,77.02,36,0,4.61,SA,1722913432
288,288,Vilhena,-12.7406,-60.1458,70.56,41,0,5.01,BR,1722913457


### Step 3: Create a new DataFrame called `hotel_df`.

In [20]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
68,Remire-Montjoly,GF,4.9167,-52.2667,100,
104,Moranbah,AU,-22.0016,148.0466,40,
219,Valguarnera Caropepe,IT,37.4953,14.3903,81,
225,Ha'Il,SA,27.5219,41.6907,36,
288,Vilhena,BR,-12.7406,-60.1458,41,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [29]:
# Set parameters to search for a hotel
radius = 10_000
params = dict(
    limit=1,
    apiKey=geoapify_key,
    categories="accommodation.hotel",
)

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude, longitude = row["Lat"], row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = f"No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Remire-Montjoly - nearest hotel: Complexe Belova
Moranbah - nearest hotel: No hotel found
Valguarnera Caropepe - nearest hotel: Hotel Villa Giulia
Ha'Il - nearest hotel: فندق كلاسيك
Vilhena - nearest hotel: No hotel found
Concepcion - nearest hotel: Marcus Hotel
Gurupi - nearest hotel: Via Norte
Saint Albans - nearest hotel: Sleep Inn
La Foa - nearest hotel: Oasis de Tendea
Aripuana - nearest hotel: No hotel found
Geroskipou - nearest hotel: Pafiana
Arish - nearest hotel: سينا
Aricak - nearest hotel: No hotel found
Mount Isa - nearest hotel: Ibis Styles


,City,Country,Lat,Lng,Humidity,Hotel Name
68,Remire-Montjoly,GF,4.9167,-52.2667,100,Complexe Belova
104,Moranbah,AU,-22.0016,148.0466,40,No hotel found
219,Valguarnera Caropepe,IT,37.4953,14.3903,81,Hotel Villa Giulia
225,Ha'Il,SA,27.5219,41.6907,36,فندق كلاسيك
288,Vilhena,BR,-12.7406,-60.1458,41,No hotel found
386,Concepcion,BO,-16.4333,-60.9000,51,Marcus Hotel
432,Gurupi,BR,-11.7292,-49.0686,40,Via Norte
437,Saint Albans,US,38.3857,-81.8362,82,Sleep Inn
476,La Foa,NC,-21.7108,165.8276,52,Oasis de Tendea
483,Aripuana,BR,-9.1667,-60.6333,43,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [30]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1.00,
    color = "City",
    hover_cols = ["Hotel Name", "Country"],
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)